In [ ]:
# On peut pas faire comme max si lower_agent > 0 (ici 1), du coup en prenant aléatoire à chaque tour, vers la fin, on peut pas respecter [1,m] à coup sur
start = time.time()
# ----- Création du modèle -----
model = Model("assignment1")
# Pour éviter d'avoir trop d'output
model.Params.OutputFlag = 0

# Variables :
x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="x")
b = model.addVars(n, n, vtype=GRB.CONTINUOUS, lb=0, name="b")
r = model.addVars(n, vtype=GRB.CONTINUOUS, name="r")

# Contraintes pour chaque agent (la somme des x de chaque agent doit être comprise entre lower_agent et upper_agent)
#for i in range(n):
#    model.addConstr(quicksum(x[i, j] for j in range(m)) >= lower_agent[i],
#                    name=f"agent_lower_{i}")
#    model.addConstr(quicksum(x[i, j] for j in range(m)) <= upper_agent[i],
#                    name=f"agent_upper_{i}")

# Contraintes pour chaque item (chaque item doit être assigné exactement à 1 agent)
for j in range(m):
    model.addConstr(quicksum(x[i, j] for i in range(n)) >= lower_item[j],
                    name=f"item_lower_{j}")
    model.addConstr(quicksum(x[i, j] for i in range(n)) <= upper_item[j],
                    name=f"item_upper_{j}")

# Contrainte liant r et b aux coûts d'assignation
for i in range(n):
    for k in range(n):
        model.addConstr(r[k] + b[i, k] >= quicksum(c[i, j] * x[i, j] for j in range(m)),
                        name=f"c3_{i}_{k}")

# Construction de l'objectif (il est re-définie à chaque itération)
#obj_expr = (quicksum(w_prime[k] * (k + 1) * r[k] for k in range(n)) + quicksum(b[i, k] for i in range(n) for k in range(n)))
obj_expr = quicksum(w_prime[k]*((k+1)*r[k] + quicksum(b[i,k] for i in range(n))) for k in range(n))
model.setObjective(obj_expr, GRB.MINIMIZE)

# ----- Arrondi Itératif -----
# À chaque itération, on résout le LP, puis on fixe le x[i,j] non encore fixée (c'est-à-dire dont lb < ub)
# qui possède la plus grande valeur (le plus grand x_{ij} dans la solution courante) en le contraignant à 1.
iteration = 0
max_iterations = n * m # borne de sécurité
#time_tot = 0
#somme_1 = [0 for i in range(n)]
somme_2 = [0 for j in range(m)]

while iteration < max_iterations:
    model.optimize()

    #print("Variables restantes :", non_fixed_vars)

    #if (iteration == 1):
    #    model.write("model1.lp")

    if model.status != GRB.OPTIMAL:
        print("Modèle pas optimal ou infaisable")
        break

    non_fixed_vars = [(i, j) for i in range(n) for j in range(m) if x[i, j].LB != x[i, j].UB]
    if not non_fixed_vars:
        print("Toutes les variables sont fixées.")
        break

    #for j in range(m):
    #    for i in range(n):
    #        print(j, i, x[i,j].X)

    sel_i, sel_j = random.choice(non_fixed_vars)
    val_modif = x[sel_i, sel_j].X

    # Fixe la variable à 1
    x[sel_i, sel_j].lb = 1
    x[sel_i, sel_j].ub = 1
    model.update()
    ##print(f"Itération {iteration}: x[{sel_i},{sel_j}] fixé à 1 (valeur LP = {val_modif})")

    # Mise à jour des compteurs
    #somme_1[sel_i] += 1
    somme_2[sel_j] += 1

    # Fixer à 0 les autres variables si borne atteinte
    #if somme_1[sel_i] >= upper_agent[sel_i]:
    #    for j in range(m):
    #        if x[sel_i, j].LB == 0:
    #            x[sel_i, j].ub = 0
    if somme_2[sel_j] >= upper_item[sel_j]:
        for i in range(n):
            if x[i, sel_j].LB == 0:
                x[i, sel_j].ub = 0
    model.update()

    #print("somme_1 =", somme_1)
    #print("somme_2 =", somme_2)

    iteration += 1

end = time.time()

# ----- Affichage de la solution -----
#mod_approx_rdm = model.objVal
#print()
#print("RUNTIME (en s) :", time_tot)
time_rdm = end-start
print("time_rdm :", time_rdm)
print('Valeur de la fonction objectif: %f' % mod_approx_rdm)
#solution = {(i, j): x[i, j].X for i in range(n) for j in range(m)}
#print("Solution d'assignation:")
#for i in range(n):
#    row = [x[i, j].X for j in range(m)]
#    print(f"Agent {i}: {row}")

##z = [sum(int(c[i,j])*x[i, j].X for j in range(m)) for i in range(n)]
##print("Vecteur z :", z)
#total_cost = sum(c[i][j] * x[i, j].X for i in range(n) for j in range(m))
##total_cost = sum(z)
##print("Coût total =", total_cost)